In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from lib import app
from lib.utils import load_points, save_points, load_json, save_json

plt.style.use(os.path.join("..", "configs", "mplstyle.yaml"))

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
int_path = "/Users/zico/Downloads/cali_int"
ext_cam1_path = "/Users/zico/Downloads/cali_ext/601"
ext_cam2_path = "/Users/zico/Downloads/cali_ext/602"

In [ ]:
def convert_points(json_paths: list, out_fname: str):
    data = load_json(json_paths[0])
    output = {
        "board_shape": [9, 1],
        "board_square_len": 0,
        "camera_resolution": [data["imageWidth"], data["imageHeight"]],
        "points": {}
    }
    for fname in json_paths:
        data = load_json(fname)
        assert len(data["shapes"]) == 9, f"File {fname} did not produce enough points"
        output["points"][data["imagePath"]] = []
        for i, p in enumerate(data["shapes"]):
            assert f"p{i+1}" == p["label"]
            output["points"][data["imagePath"]].append(p["points"][0])

    save_json(out_fname, output)

In [ ]:
# Convert labelme points to acinoset points.
# Convert intrinsic points.
json_paths = sorted(glob(os.path.join(int_path, '*.json')))
convert_points(json_paths, os.path.join(int_path, "output", "points.json"))

# Convert extrinsic points.
json_paths = sorted(glob(os.path.join(ext_cam1_path, '*.json')))
convert_points(json_paths, os.path.join(os.path.dirname(ext_cam1_path), "points", "points1.json"))

json_paths = sorted(glob(os.path.join(ext_cam2_path, '*.json')))
convert_points(json_paths, os.path.join(os.path.dirname(ext_cam2_path), "points", "points2.json"))

In [ ]:
# Intrinsic calibration.
K, D, R, t, used_points = app.calibrate_arabia_intrinsics(os.path.join(int_path, "output", "points.json"), os.path.join(int_path, "output", "camera.json"))


In [ ]:
# Extrinsic calibration.
points_fpaths = sorted(glob(os.path.join("/Users/zico/Downloads/cali_ext", "points", "points[1-4].json")))
scene_fpath = os.path.join("/Users/zico/Downloads/cali_ext", f"{len(points_fpaths)}_cam_scene.json")
app.calibrate_arabia_extrinsics(scene_fpath, points_fpaths[0], points_fpaths[1], os.path.join(int_path, "output", "camera.json"))

In [ ]:
# Perform SBA on calibration points to refine extrinsic calibration parameters.
scene_sba_fpath = scene_fpath.replace('.json','_sba.json')

res = app.sba_board_points_standard(
    scene_fpath, points_fpaths, out_fpath=scene_sba_fpath,
)

plt.plot(res['before'], label='Cost before')
plt.plot(res['after'], label='Cost after')
plt.legend()
plt.show(block=False)